![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Adverse Drug Events Detection using Named Entity Recognition, Classification and Assertion Status Models

`ADE NER`: Extracts ADE and DRUG entities from clinical texts.

`ADE Classifier`: CLassify if a sentence is ADE-related (`True`) or not (`False`)

We use several datasets to train these models:

- Twitter dataset, which is used in paper "`Deep learning for pharmacovigilance: recurrent neural network architectures for labeling adverse drug reactions in Twitter posts`" (https://pubmed.ncbi.nlm.nih.gov/28339747/)
- ADE-Corpus-V2, which is used in paper "`An Attentive Sequence Model for Adverse Drug Event Extraction from Biomedical Text`" (https://arxiv.org/abs/1801.00625) and availe online: https://sites.google.com/site/adecorpus/home/document.
- CADEC dataset, which is sued in paper `Cadec: A corpus of adverse drug event annotations` (https://pubmed.ncbi.nlm.nih.gov/25817970)

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
#from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.4
Spark NLP_JSL Version : 4.2.3
Out[81]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.139.64.5:7077 
 AppName 
 Databricks Shell

## ADE Classifier 

ADE Classifier Classifies if a sentence is ADE-related (`True`) or not (`False`)

`True` : The sentence is talking about a possible ADE

`False` : The sentences doesn't have any information about an ADE.


|index |model |Predicted Entities|
|-----:|:-----|:----------------:|
| 1| [classifierdl_ade_biobert](https://nlp.johnsnowlabs.com/2021/01/21/classifierdl_ade_biobert_en.html)   |True, False|
| 2| [classifierdl_ade_clinicalbert](https://nlp.johnsnowlabs.com/2021/01/21/classifierdl_ade_clinicalbert_en.html)  |True, False|
| 3| [classifierdl_ade_conversational_biobert](https://nlp.johnsnowlabs.com/2021/01/21/classifierdl_ade_conversational_biobert_en.html)  |True, False|
| 4| [bert_sequence_classifier_ade](https://nlp.johnsnowlabs.com/2022/02/08/bert_sequence_classifier_ade_en.html)  |True, False|
| 5| [bert_sequence_classifier_ade_augmented](https://nlp.johnsnowlabs.com/2022/07/27/bert_sequence_classifier_ade_augmented_en_3_0.html)   |ADE, noADE|
| 6| [distilbert_sequence_classifier_ade](https://nlp.johnsnowlabs.com/2022/02/08/distilbert_sequence_classifier_ade_en.html)  |True, False|

### ADE Classifier with BioBert

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

bert_embeddings = BertEmbeddings.pretrained("biobert_pubmed_base_cased")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")\
    .setMaxSentenceLength(512)

embeddingsSentence = SentenceEmbeddings() \
    .setInputCols(["sentence", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")\
    .setStorageRef('biobert_pubmed_base_cased')

classsifierdl = ClassifierDLModel.pretrained("classifierdl_ade_biobert", "en", "clinical/models")\
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class")

ade_clf_pipeline = Pipeline(
    stages=[documentAssembler, 
            tokenizer,
            bert_embeddings,
            embeddingsSentence,
            classsifierdl])


empty_data = spark.createDataFrame([[""]]).toDF("text")
ade_clf_model = ade_clf_pipeline.fit(empty_data)

ade_lp_pipeline = LightPipeline(ade_clf_model)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[ | ][OK!]
classifierdl_ade_biobert download started this may take some time.
Approximate size to download 21.8 MB
[ | ][OK!]

In [0]:
text = """I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.
I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication."""

print(ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].result)
print(ade_lp_pipeline.fullAnnotate(text)[0]["class"])

True
[Annotation(category, 0, 247, True, {'sentence': '0', 'False': '0.012636117', 'True': '0.9873639'})]

In [0]:
text="I just took an Advil and have no gastric problems so far."

print(ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].result)
print(ade_lp_pipeline.fullAnnotate(text)[0]["class"])

False
[Annotation(category, 0, 56, False, {'sentence': '0', 'False': '0.9999769', 'True': '2.3126835E-5'})]

As you see `gastric problems` is not detected as `ADE` as it is in a negative context. So, classifier did a good job detecting that.

In [0]:
text="""Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. 
I had every test in the book done at the hospital, and they couldn't find anything. I was completley healthy! I am thinking it was from the voltaren. 
I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. 
I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead."""

print(ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].result)
print(ade_lp_pipeline.fullAnnotate(text)[0]["class"])

True
[Annotation(category, 0, 567, True, {'sentence': '0', 'False': '0.0146723185', 'True': '0.9853277'})]

In [0]:
ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].metadata["True"]

Out[86]: '0.9853277'

In [0]:
texts = ["The patient was prescribed 1000 mg fish oil and multivitamins. She was discharged on zopiclone and ambrisentan.",
"I feel a bit drowsy & have a little blurred vision, after taking a pill.",
"I've been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it.",
"Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes.",
"So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50."]

for text in texts:

  result = ade_lp_pipeline.annotate(text)
  cls = ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].result
  confidence = ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].metadata[cls]
  print (cls,"\t",confidence)


False 0.9999689
True 0.7971171
False 0.99999654
True 0.50409865
False 0.9763144

### ADE Classifier trained with conversational (short) sentences

This model is trained on short, conversational sentences related to ADE and is supposed to do better on the text that is short and used in a daily context.

In [0]:
conv_classsifierdl = ClassifierDLModel.pretrained("classifierdl_ade_conversational_biobert", "en", "clinical/models")\
            .setInputCols(["sentence_embeddings"]) \
            .setOutputCol("class")

conv_ade_clf_pipeline = Pipeline(
    stages=[documentAssembler, 
            tokenizer,
            bert_embeddings,
            embeddingsSentence,
            conv_classsifierdl])

empty_data = spark.createDataFrame([[""]]).toDF("text")
conv_ade_clf_model = conv_ade_clf_pipeline.fit(empty_data)

conv_ade_lp_pipeline = LightPipeline(conv_ade_clf_model)

classifierdl_ade_conversational_biobert download started this may take some time.
Approximate size to download 21.8 MB
[ | ][OK!]

In [0]:
text = "after taking a pill, he denies any pain"

conv_ade_lp_pipeline.annotate(text)['class'][0]

print(conv_ade_lp_pipeline.fullAnnotate(text)[0]["class"][0].result)
print(conv_ade_lp_pipeline.fullAnnotate(text)[0]["class"])

False
[Annotation(category, 0, 38, False, {'sentence': '0', 'False': '0.95695204', 'True': '0.043047957'})]

### ADE Sequence Classifier

MedicalBertForSequenceClassification

In [0]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
        
sequenceClassifier = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_ade_augmented", "en", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class")
    

ade_clf_pipeline = Pipeline(stages=[
                    document_assembler, 
                    tokenizer,
                    sequenceClassifier])


data = spark.createDataFrame([["So glad I am off effexor, so sad it ruined my teeth. tip Please be carefull taking antideppresiva and read about it 1st"],
                              ["Religare Capital Ranbaxy has been accepting approval for Diovan since 2012"]]).toDF("text")
              
result = ade_clf_pipeline.fit(data).transform(data)

result.select("text", "class.result").show(truncate=False)

bert_sequence_classifier_ade_augmented download started this may take some time.
[ | ][OK!]
+-----------------------------------------------------------------------------------------------------------------------+-------+
text |result |
+-----------------------------------------------------------------------------------------------------------------------+-------+
So glad I am off effexor, so sad it ruined my teeth. tip Please be carefull taking antideppresiva and read about it 1st|[ADE] |
Religare Capital Ranbaxy has been accepting approval for Diovan since 2012 |[noADE]|
+-----------------------------------------------------------------------------------------------------------------------+-------+

MedicalDistilBertForSequenceClassification

In [0]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
    

sequenceClassifier = MedicalDistilBertForSequenceClassification.pretrained("distilbert_sequence_classifier_ade", "en", "clinical/models")\
    .setInputCols(["document","token"])\
    .setOutputCol("class")
 
ade_clf_pipeline = Pipeline(stages=[
                    document_assembler, 
                    tokenizer,
                    sequenceClassifier])


data = spark.createDataFrame([["I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication."],
                              ["Religare Capital Ranbaxy has been accepting approval for Diovan since 2012"]]).toDF("text")
              
result = ade_clf_pipeline.fit(data).transform(data)

result.select("text", "class.result").show(truncate=100)

distilbert_sequence_classifier_ade download started this may take some time.
[ | ][OK!]
+----------------------------------------------------------------------------------------------------+-------+
 text| result|
+----------------------------------------------------------------------------------------------------+-------+
I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numb...| [True]|
 Religare Capital Ranbaxy has been accepting approval for Diovan since 2012|[False]|
+----------------------------------------------------------------------------------------------------+-------+

## ADE NER

Extracts `ADE` and `DRUG` entities from text.

|    | model_name                 |Predicted Entities|
|---:|:---------------------------|:----------------:|
|  1 | [ner_ade_clinical](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_clinical_en.html)      |ADE, DRUG|
|  2 | [ner_ade_biobert](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_biobert_en.html)        |ADE, DRUG|
|  3 | [ner_ade_healthcare](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_healthcare_en.html)  |ADE, DRUG|
|  4 | [ner_ade_clinicalbert](https://nlp.johnsnowlabs.com/2021/04/01/ner_ade_clinicalbert_en.html)   |ADE, DRUG|
|  5 | [bert_token_classifier_ner_ade](https://nlp.johnsnowlabs.com/2022/01/04/bert_token_classifier_ner_ade_en.html) |ADE, DRUG|
|  6 | [bert_token_classifier_ade_tweet_binary](https://nlp.johnsnowlabs.com/2022/07/29/bert_token_classifier_ade_tweet_binary_en_3_0.html) |ADE |

#### ADE NER with Word embeddings

In [0]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

ade_ner = MedicalNerModel.pretrained("ner_ade_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverterInternal() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    ade_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_model = ner_pipeline.fit(empty_data)

ade_ner_lp = LightPipeline(ade_ner_model)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_ade_clinical download started this may take some time.
[ | ][OK!]

In [0]:
text = """I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.
I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication."""

light_result = ade_ner_lp.fullAnnotate(text)

chunks = []
entities = []
begin =[]
end = []
confidence = []

for n in light_result[0]['ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    confidence.append(n.metadata['confidence'])  

import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                    'begin': begin, 'end': end, "confidence":confidence})

df

Out[93]:

,chunks,entities,begin,end,confidence
0,allergic reaction,ADE,10,26,0.68645
1,vancomycin,DRUG,31,40,0.9988
2,itchy skin,ADE,52,61,0.71365
3,sore throat/burning/itching,ADE,64,90,0.81229997
4,numbness of tongue,ADE,93,110,0.6829667
5,gums,ADE,116,119,0.7055


As you see `gastric problems` is not detected as `ADE` as it is in a negative context. So, NER did a good job ignoring that.

#### ADE NER with Bert embeddings

In [0]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

bert_embeddings = BertEmbeddings.pretrained("biobert_pubmed_base_cased")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")
  
ade_ner_bert = MedicalNerModel.pretrained("ner_ade_biobert", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverterInternal() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    bert_embeddings,
    ade_ner_bert,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_model_bert = ner_pipeline.fit(empty_data)

ade_ner_lp_bert = LightPipeline(ade_ner_model_bert)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[ | ][OK!]
ner_ade_biobert download started this may take some time.
[ | ][OK!]

In [0]:
text = """I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.
I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication."""

light_result = ade_ner_lp_bert.fullAnnotate(text)

chunks = []
entities = []
begin =[]
end = []
confidence = []

for n in light_result[0]['ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    confidence.append(n.metadata['confidence'])  

import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                    'begin': begin, 'end': end, "confidence":confidence})

df

Out[95]:

,chunks,entities,begin,end,confidence
0,allergic reaction,ADE,10,26,0.8071
1,vancomycin,DRUG,31,40,0.9979
2,itchy skin,ADE,52,61,0.94315004
3,sore throat/burning/itching,ADE,64,90,0.90735
4,numbness of tongue and gums,ADE,93,119,0.90212
5,any other medication,DRUG,227,246,0.75869995


#### ADE NER for Tweets

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols("sentence")\
    .setOutputCol("token")

tokenClassifier = MedicalBertForTokenClassifier.pretrained("bert_token_classifier_ade_tweet_binary", "en", "clinical/models")\
    .setInputCols("token", "sentence")\
    .setOutputCol("ner")\
    .setCaseSensitive(True)

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")


nlpPipeline =  Pipeline(stages=[
                      documentAssembler,
                      sentenceDetector,
                      tokenizer,
                      tokenClassifier,
                      ner_converter])

ade_ner_model_tweet= nlpPipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

ade_ner_lp_tweet = LightPipeline(ade_ner_model_tweet)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[ | ][OK!]
bert_token_classifier_ade_tweet_binary download started this may take some time.
[ | ][OK!]

In [0]:
twitter_text = """I understand you very well. :( just got 1st urgh ! humira worked for me for just 3months then got painful reactions.
This vyvanse got me sweating right now and i dont even know why!,
Wonder which drug is doing this memory lapse thing. My guess the Duloxetine.
I used to be on paxil but that made me more depressed and prozac made me angry.
Maybe it's because of the effect of seroquel, but when I eat fast carbohydrates, I feel the sugar drop."""


In [0]:
light_result = ade_ner_lp_tweet.fullAnnotate(twitter_text)

chunks = []
entities = []
begin =[]
end = []
confidence = []

for n in light_result[0]['ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    confidence.append(n.metadata['confidence'])  

import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                    'begin': begin, 'end': end, "confidence":confidence})

df

Out[98]:

,chunks,entities,begin,end,confidence
0,painful reactions,ADE,98,114,0.96369493
1,sweating,ADE,137,144,0.99936104
2,memory lapse,ADE,215,226,0.98965096
3,depressed,ADE,304,312,0.99975455
4,angry,ADE,333,337,0.99960786
5,sugar drop,ADE,432,441,0.90284485


## ADE NER with AssertionDL Model

In [0]:
assertion_ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ass_ner_chunk")\
    .setWhiteList(['ADE'])

biobert_assertion = AssertionDLModel.pretrained("assertion_dl_biobert", "en", "clinical/models") \
    .setInputCols(["sentence", "ass_ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

assertion_ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    bert_embeddings,
    ade_ner_bert,
    ner_converter,
    assertion_ner_converter,
    biobert_assertion])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ass_ner_model_bert = assertion_ner_pipeline.fit(empty_data)

ade_ass_ner_model_lp_bert = LightPipeline(ade_ass_ner_model_bert)

assertion_dl_biobert download started this may take some time.
[ | ][OK!]

In [0]:
import pandas as pd
text = "I feel a bit drowsy & have a little blurred vision, so far no gastric problems. I have been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it. Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes. So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50."

print (text)

light_result = ade_ass_ner_model_lp_bert.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]
confidence=[]


for n,m in zip(light_result['ass_ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
    confidence.append(n.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, "confidence":confidence})

df

I feel a bit drowsy & have a little blurred vision, so far no gastric problems. I have been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it. Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes. So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50.
Out[100]:

,chunks,entities,assertion,confidence
0,drowsy,ADE,present,0.7226
1,blurred vision,ADE,present,0.9296
2,gastric problems,ADE,absent,0.8494
3,pains,ADE,present,0.8005


Looks great ! `gastric problems` is detected as `ADE` and `absent`

## ADE Relation Extraction Model

ADE Relation Extraction classifies if an adverse event is caused by a drug or not. 1: Shows the adverse event and drug entities are related, 0 : Shows the adverse event and drug entities are not related

|    | model_name                 |Predicted Entities|
|---:|:---------------------------|:----------------:|
|  1 | [re_ade_clinical](https://nlp.johnsnowlabs.com/2021/07/12/re_ade_clinical_en.html)    |0, 1|
|  2 | [re_ade_biobert](https://nlp.johnsnowlabs.com/2021/07/16/re_ade_biobert_en.html)      |0, 1|
|  3 | [redl_ade_biobert](https://nlp.johnsnowlabs.com/2021/07/12/redl_ade_biobert_en.html)  |0, 1|
|  4 | [re_ade_conversational](https://nlp.johnsnowlabs.com/2022/07/27/re_ade_conversational_en_3_0.html)  |not_related, is_related|

In [0]:
# get relations in a pandas dataframe
import pandas as pd

def get_relations_df (results, rel_col='relations', chunk_col='ner_chunks'):
    rel_pairs=[]
    chunks = []

    for rel in results[rel_col]:
        rel_pairs.append((
            rel.metadata['entity1_begin'],
            rel.metadata['entity1_end'],
            rel.metadata['chunk1'], 
            rel.metadata['entity1'], 
            rel.metadata['entity2_begin'],
            rel.metadata['entity2_end'],
            rel.metadata['chunk2'], 
            rel.metadata['entity2'],
            rel.result, 
            rel.metadata['confidence'],
        ))

    for chunk in results[chunk_col]:
        chunks.append((
            chunk.metadata["sentence"],
            chunk.begin,
            chunk.end,
            chunk.result, 
        ))

    rel_df = pd.DataFrame(rel_pairs, columns=['entity1_begin', 'entity1_end', 'chunk1', 'entity1', 'entity2_begin', 'entity2_end', 'chunk2', 'entity2', 'relation', 'confidence'])

    chunks_df = pd.DataFrame(chunks, columns = ["sentence", "begin", "end", "chunk"])
    chunks_df.begin = chunks_df.begin.astype(str)
    chunks_df.end = chunks_df.end.astype(str)

    result_df = pd.merge(rel_df,chunks_df, left_on=["entity1_begin", "entity1_end", "chunk1"], right_on=["begin", "end", "chunk"])[["sentence"] + list(rel_df.columns)]


    return result_df

In [0]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

words_embedder = WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos_tags")

ner_tagger = MedicalNerModel()\
    .pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "token", "embeddings")\
    .setOutputCol("ner_tags")    

ner_chunker = NerConverterInternal()\
    .setInputCols(["sentence", "token", "ner_tags"])\
    .setOutputCol("ner_chunks")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos_tags", "token"])\
    .setOutputCol("dependencies")

reModel = RelationExtractionModel()\
    .pretrained("re_ade_clinical", "en", 'clinical/models')\
    .setInputCols(["embeddings", "pos_tags", "ner_chunks", "dependencies"])\
    .setOutputCol("relations")\
    .setMaxSyntacticDistance(20)\
    .setRelationPairs(["drug-ade, ade-drug"])\
    .setRelationPairsCaseSensitive(False)\
    .setCustomLabels({"1": "is_related", "0": "not_related"})

pipeline = Pipeline(stages=[
    documentAssembler,
    sentenceDetector,
    tokenizer, 
    words_embedder, 
    pos_tagger, 
    ner_tagger,
    ner_chunker,
    dependency_parser,
    reModel
])

empty_data = spark.createDataFrame([[""]]).toDF("text")
ade_model = pipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[ | ][OK!]
ner_ade_clinical download started this may take some time.
[ | ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][OK!]
re_ade_clinical download started this may take some time.
Approximate size to download 10.9 MB
[ | ][OK!]

In [0]:
text = ["""
Hypersensitivity to aspirin can be manifested as acute asthma, urticaria and/or angioedema, or a systemic anaphylactoid reaction.
A patient had undergone a renal transplantation as a result of malignant hypertension, and immunosuppressive therapy consisting of cyclosporin and prednisone ,  developed  sweating  and  thrombosis alone 5 years following the transplantation but there were not stomach pain.  
A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for   rheumatoid arthritis  presented  with  tense bullae and cutaneous fragility on the face and the back of the hands.""",

"""We describe the side effects of 5-FU in a colon cancer patient who suffered severe mucositis,  prolonged myelosuppression, and neurologic toxicity that required admission to the intensive care unit who  has a healthy appetite.
The reported cases of in utero exposure to cyclosposphamide shared the following manifestations with our patient who suffered  growth deficiency, developmental delay, craniosynostosis, blepharophimosis, flat nasal bridge and abnormal ears.
I have an allergic reaction to vancomycin so I have itchy skin, sore throat/burning/itching, numbness of tongue and gums.I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.
I experienced fatigue, muscle cramps, anxiety, agression and sadness after taking Lipitor but no more adverse after passing Zocor.
A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands.""" ]

In [0]:
light_results = LightPipeline(ade_model).fullAnnotate(text)

In [0]:
get_relations_df(light_results[0], 'relations')

Out[105]:

,sentence,entity1_begin,entity1_end,chunk1,entity1,entity2_begin,entity2_end,chunk2,entity2,relation,confidence
0,0,21,27,aspirin,DRUG,50,61,acute asthma,ADE,is_related,1.0
1,0,21,27,aspirin,DRUG,64,72,urticaria,ADE,is_related,1.0
2,0,21,27,aspirin,DRUG,81,90,angioedema,ADE,is_related,1.0
3,0,21,27,aspirin,DRUG,98,128,systemic anaphylactoid reaction,ADE,is_related,0.999949
4,1,262,272,cyclosporin,DRUG,303,310,sweating,ADE,is_related,0.99999905
5,1,262,272,cyclosporin,DRUG,318,327,thrombosis,ADE,is_related,1.0
6,1,278,287,prednisone,DRUG,303,310,sweating,ADE,is_related,1.0
7,1,278,287,prednisone,DRUG,318,327,thrombosis,ADE,is_related,1.0
8,2,433,440,naproxen,DRUG,550,561,tense bullae,ADE,is_related,1.0
9,2,433,440,naproxen,DRUG,567,623,cutaneous fragility on the face and the back of the hands,ADE,is_related,0.99925286


In [0]:
from sparknlp_display import RelationExtractionVisualizer

vis = RelationExtractionVisualizer()
re_vis = vis.display(light_results[0], 'relations', show_relations=True,return_html=True) # default show_relations: True
displayHTML(re_vis)

/databricks/python/lib/python3.8/site-packages/sparknlp_display/relation_extraction.py:366: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
 relation_coordinates = np.array(relation_coordinates)

Hypersensitivity to aspirin DRUG can be manifested as acute asthma ADE , urticaria ADE and/or angioedema ADE , or a systemic anaphylactoid reaction ADE . A patient had undergone a renal transplantation as a result of malignant hypertension, and immunosuppressive therapy consisting of cyclosporin DRUG and prednisone DRUG , developed sweating ADE and thrombosis ADE alone 5 years following the transplantation but there were not stomach pain. A 44-year-old man taking naproxen DRUG for chronic low back pain and a 20-year-old woman on oxaprozin DRUG for rheumatoid arthritis presented with tense bullae ADE and cutaneous fragility on the face and the back of the hands ADE . is_related is_related <polyline fill="none" points="649.0,355.0 651.4795918367346,353.7960798646822 653.9591836734694,352.59312871337613 656.4387755102041,351.3921155300938 658.9183673469388,350.19400929884654 661.3979591836735,348.99977900364644 663.8775510204081,347.81039362850504 666.357142857143,346.6268221574345 668.8367346938776,345.4500335744461 671.3163265306123,344.2809968635519 673.7959183673469,343.1206810087633 676.2755102040815,341.97005499409255 678.7551020408163,340.8300878035512 681.234693877551,339.701748421151 683.7142857142857,338.5860058309038 686.1938775510205,337.4838290168213 688.6734693877552,336.3961869629151 691.1530612244899,335.3240486531973 693.6326530612245,334.2683830716793 696.1122448979592,333.23015920237316 698.5918367346937,332.21034602929046 701.0714285714284,331.20991253644314 703.5510204081633,330.22982770784284 706.0306122448981,329.2710605275013 708.5102040816327,328.3345799794304 710.9897959183673,327.4213550476417 713.4693877551019,326.5323547161472 715.9489795918366,325.6685479689586 718.4285714285716,324.8309037900875 720.9081632653061,324.0203911635458 723.3877551020407,323.2379790733453 725.8673469387754,322.4846365034977 728.3469387755101,321.7613324380148 730.8265306122449,321.06903586090834 733.3061224489795,320.40871575619 735.7857142857142,319.78134110787175 738.265306122449,319.1878808999652 740.7448979591836,318.6293041164821 743.2244897959183,318.1065797414343 745.704081632653,317.6206767588335 748.1836734693877,317.1725641526915 750.6632653061224,316.76321090702004 753.1428571428571,316.3935860058309 755.6224489795918,316.06465843313583 758.1020408163265,315.7773971729466 760.5816326530612,315.5327712092751 763.0612244897959,315.33174952613285 765.5408163265306,315.1753011075317 768.0204081632653,315.0643949374835 770.5,315.0 770.5,315.0 772.9795918367346,314.9827622844222 775.4591836734692,315.0120358014093 777.9387755102041,315.0868515669492 780.4183673469387,315.20624059703016 782.8979591836735,315.3692339076405 785.3775510204081,315.57486251476854 787.857142857143,315.82215743440236 790.3367346938776,316.1101496825303 792.8163265306124,316.4378702751404 795.2959183673469,316.8043502282212 797.7755102040816,317.2086205577607 800.2551020408162,317.6497122797473 802.7346938775509,318.12665641016923 805.2142857142857,318.6384839650146 807.6938775510206,319.1842259602717 810.1734693877553,319.76291341192865 812.6530612244899,320.37357733597395 815.1326530612245,321.01524874839566 817.6122448979592,321.686958665182 820.0918367346937,322.38773810232124 822.5714285714286,323.1166180758017 825.0510204081633,323.87262960161155 827.530612244898,324.65480369573896 830.0102040816327,325.46217137417227 832.4897959183675,326.29376365289966 834.9693877551019,327.14861154790935 837.4489795918367,328.0257460751897 839.9285714285714,328.92419825072886 842.4081632653063,329.842999090515 844.8877551020408,330.78117961053636 847.3673469387757,331.73777082678134 849.8469387755101,332.711803755238 852.3265306122449,333.7023094118947 854.8061224489795,334.70831881273955 857.2857142857144,335.72886297376095 859.765306122449,336.7629729109469 862.2448979591836,337.80967964028594 864.7244897959183,338.86801417776604 867.204081632653,339.9370075393756 869.6836734693877,341.01569074110273 872.1632653061224,342.1030947989358 874.64285714

## ADE models applied to Spark Dataframes

In [0]:
import pyspark.sql.functions as F

! wget -q	https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/sample_ADE_dataset.csv

dbutils.fs.cp("file:/databricks/driver/sample_ADE_dataset.csv", "dbfs:/")

Out[107]: True

In [0]:
ade_DF = spark.read\
                .option("header", "true")\
                .csv("/sample_ADE_dataset.csv")\
                .filter(F.col("label").isin(['True','False']))

ade_DF.show(truncate=50)

+--------------------------------------------------+-----+
 text|label|
+--------------------------------------------------+-----+
Do U know what Meds are R for bipolar depressio...|False|
# hypercholesterol: Because of elevated CKs (pe...| True|
Her weight, respirtory status and I/O should be...|False|
* DM - Pt had several episodes of hypoglycemia ...| True|
We report the case of a female acromegalic pati...| True|
2 . Calcipotriene 0.005% Cream Sig: One (1) App...|False|
Always tired, and possible blood clots. I was o...| True|
A difference in chemical structure between thes...|False|
10 . She was left on prednisone 20mg qd due to ...|False|
The authors suggest that risperidone may increa...| True|
- Per oral maxillofacial surgery there is no ev...|False|
@marionjross Cipro is just as bad! Stay away fr...|False|
A young woman with epilepsy had tonic-clonic se...| True|
Intravenous methotrexate is an effective adjunc...|False|
PURPOSE: To report new indocyanine green angiog...|False|
2 . Docusate Sodium 50 mg/5 mL Liquid [**Hospit...|False|
 consider neupogen.|False|
He was treated allopurinol and Rasburicase for ...|False|
Toxicity, pharmacokinetics, and in vitro hemodi...| True|
# thrombocytopenia: Secondary to chemotherapy a...| True|
+--------------------------------------------------+-----+
only showing top 20 rows

**With BioBert version of NER** (will be slower but more accurate)

In [0]:
import pyspark.sql.functions as F

ner_converter = NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(['ADE'])

ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    bert_embeddings,
    ade_ner_bert,
    ner_converter])


empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_model = ner_pipeline.fit(empty_data)

result = ade_ner_model.transform(ade_DF)

sample_df = result.select('text','ner_chunk.result')\
                  .toDF('text','ADE_phrases')\
                  .filter(F.size('ADE_phrases')>0).toPandas()

In [0]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [0]:
sample_df.sample(20)

Out[111]:

,text,ADE_phrases
1,* DM - Pt had several episodes of hypoglycemia on lantus due to decreasing oral intake.,[hypoglycemia]
26,"With itraconazole, hepatotoxic reactions have only very rarely been reported, and histologic data are lacking.",[hepatotoxic reactions]
14,"A transient tonic pupillary response, denervation supersensitivity, and abnormal visual-evoked potentials in quinine toxicity, to our knowledge, have not been previously reported.","[transient tonic pupillary response, denervation supersensitivity, abnormal visual-evoked potentials]"
21,"2 years with no problems, then toe neuropathy for two years now and other foot problems because of this I assume. I stopped Lipitor after taking it for 2 years and toe neuropathy started. I also had stomach problems and pain. After stopping Lipitor, I thought my heart would jump out of my chest, but now stomach and heart are both OK - Feet are terrible thanks to Lipitor. I know many others that have had problems with Lipitor - feet and legs - and had to stop. All my blood test were normal. I weigh 114 and 5'2 . Hike, play golf and keep busy. It is a terrible drug and should be off the market.","[toe neuropathy, foot problems, toe neuropathy, stomach problems, pain]"
5,A young woman with epilepsy had tonic-clonic seizures during antineoplastic therapy with adriamycin and cisplatin.,[tonic-clonic seizures]
12,"Effexor side effects; headaches, no appetite.","[Effexor side effects, headaches, no appetite]"
17,Hepatic reactions to cyclofenil.,[Hepatic reactions]
20,"Eight days after the end of interferon treatment, he showed signs of inability to sit still for ten seconds and walked around constantly.",[inability to sit]
7,PURPOSE: To report new indocyanine green angiographic (ICGA) findings after intravitreal bevacizumab (IVB) for myopic choroidal neovascularization (mCNV).,[indocyanine green angiographic]
11,"# Maculopapular rash: over extremities, chest and back, thought [**1-14**] zosyn which has been discontinued, mother with [**Name2 (NI) *] allergy, consider [**Name2 (NI) *] an allergy.",[Maculopapular rash]


**Doing the same with clinical embeddings version** (faster results)

In [0]:
import pyspark.sql.functions as F

ner_converter = NerConverterInternal() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")\
  .setWhiteList(['ADE'])

ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    ade_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_model = ner_pipeline.fit(empty_data)

result = ade_ner_model.transform(ade_DF)

result.select('text','ner_chunk.result')\
.toDF('text','ADE_phrases').filter(F.size('ADE_phrases')>0)\
.show(truncate=70)


+----------------------------------------------------------------------+----------------------------------------------------------------------+
 text| ADE_phrases|
+----------------------------------------------------------------------+----------------------------------------------------------------------+
# hypercholesterol: Because of elevated CKs (peaked at 819) the pat...| [elevated CKs]|
We report the case of a female acromegalic patient in whom multiple...| [multiple hepatic adenomas]|
Always tired, and possible blood clots. I was on Voltaren for about...| [blood clots that traveled to my eye, back pain]|
The authors suggest that risperidone may increase affect in patient...| [increase affect]|
A young woman with epilepsy had tonic-clonic seizures during antine...| [tonic-clonic seizures]|
Intravenous methotrexate is an effective adjunct to steroid therapy...| [dermatomyositis-polyositis]|
PURPOSE: To report new indocyanine green angiographic (ICGA) findin...| [indocyanine green angiographic (ICGA) findings]|
Toxicity, pharmacokinetics, and in vitro hemodialysis clearance of ...| [Toxicity]|
 # thrombocytopenia: Secondary to chemotherapy and MDS/AML concerns.| [thrombocytopenia]|
A fatal massive pulmonary embolus developed in a patient treated wi...| [fatal massive pulmonary embolus]|
# Maculopapular rash: over extremities, chest and back, thought [**...| [Maculopapular rash]|
 Hypokalemia after normal doses of neubulized albuterol (salbutamol).| [Hypokalemia]|
A transient tonic pupillary response, denervation supersensitivity,...|[transient tonic pupillary response, denervation supersensitivity, ...|
As per above, ID added Atovaquone for PCP [**Name9 (PRE) *] given t...| [BM suppression, liver damage]|
Electrocardiographic findings and laboratory data indicated a diagn...| [acute myocardial infarction]|
 Hepatic reactions to cyclofenil.| [Hepatic reactions]|
Therefore, parenteral amiodarone was implicated as the cause of acu...| [acute hepatitis]|
Vincristine levels were also assayed and showed a dramatic decline ...| [dramatic decline in postexchange levels]|
Eight days after the end of interferon treatment, he showed signs o...| [inability to sit]|
2 years with no problems, then toe neuropathy for two years now and...|[toe neuropathy, toe neuropathy, stomach problems, pain, heart woul...|
+----------------------------------------------------------------------+----------------------------------------------------------------------+
only showing top 20 rows

### Creating sentence dataframe (one sentence per row) and getting ADE entities and categories

In [0]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")\
      .setExplodeSentences(True)

tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

bert_embeddings = BertEmbeddings.pretrained("biobert_pubmed_base_cased")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["sentence", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")\
      .setStorageRef('biobert_pubmed_base_cased')

classsifierdl = ClassifierDLModel.pretrained("classifierdl_ade_biobert", "en", "clinical/models")\
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("class")\
      .setStorageRef('biobert_pubmed_base_cased')

ade_ner = MedicalNerModel.pretrained("ner_ade_biobert", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")
  
ner_converter = NerConverterInternal() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['ADE'])

ner_clf_pipeline = Pipeline(
    stages=[documentAssembler, 
            sentenceDetector,
            tokenizer,
            bert_embeddings,
            embeddingsSentence,
            classsifierdl,
            ade_ner,
            ner_converter])

ade_Sentences = ner_clf_pipeline.fit(ade_DF)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[ | ][OK!]
classifierdl_ade_biobert download started this may take some time.
Approximate size to download 21.8 MB
[ | ][OK!]
ner_ade_biobert download started this may take some time.
[ | ][OK!]

In [0]:
import pyspark.sql.functions as F

ade_Sentences.transform(ade_DF).select('sentence.result','ner_chunk.result','class.result')\
                               .toDF('sentence','ADE_phrases','is_ADE').show(truncate=60)

+------------------------------------------------------------+---------------------------------------------+-------+
 sentence| ADE_phrases| is_ADE|
+------------------------------------------------------------+---------------------------------------------+-------+
 [Do U know what Meds are R for bipolar depression?]| []|[False]|
 [Currently #FDA approved #quetiapine AKA #Seroquel]| []|[False]|
[# hypercholesterol: Because of elevated CKs (peaked at 8...| [elevated CKs]|[False]|
[Her weight, respirtory status and I/O should be monitore...| []|[False]|
[* DM - Pt had several episodes of hypoglycemia on lantus...| [hypoglycemia]| [True]|
[We report the case of a female acromegalic patient in wh...| [hepatic adenomas]| [True]|
 [2 .]| []|[False]|
[Calcipotriene 0.005% Cream Sig: One (1) Appl Topical [**...| []|[False]|
 [Always tired, and possible blood clots.]| [tired, blood clots]|[False]|
[I was on Voltaren for about 4 years and all of the sudde...|[stroke, blood clots that traveled to my eye]|[False]|
[I had every test in the book done at the hospital, and t...| []|[False]|
 [I was completley healthy!]| [completley healthy]|[False]|
 [I am thinking it was from the voltaren.]| []|[False]|
[I have been off of the drug for 8 months now, and have n...| []|[False]|
[I started eating healthy and working out and that has he...| []|[False]|
 [I can now sleep all thru the night.]| []|[False]|
 [I wont take this again.]| []|[False]|
 [If I have the back pain, I will pop a tylonol instead.]| []|[False]|
[A difference in chemical structure between these two dru...| []|[False]|
 [10 .]| []|[False]|
+------------------------------------------------------------+---------------------------------------------+-------+
only showing top 20 rows

## Creating a pretrained pipeline with ADE NER, Assertion and Classifer

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

bert_embeddings = BertEmbeddings.pretrained("biobert_pubmed_base_cased")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

ade_ner = MedicalNerModel.pretrained("ner_ade_biobert", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")\
      .setStorageRef('biobert_pubmed_base_cased')

ner_converter = NerConverterInternal() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

assertion_ner_converter = NerConverterInternal() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ass_ner_chunk")\
      .setWhiteList(['ADE'])

biobert_assertion = AssertionDLModel.pretrained("assertion_dl_biobert", "en", "clinical/models") \
      .setInputCols(["sentence", "ass_ner_chunk", "embeddings"]) \
      .setOutputCol("assertion")

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["sentence", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")\
      .setStorageRef('biobert_pubmed_base_cased')

classsifierdl = ClassifierDLModel.pretrained("classifierdl_ade_conversational_biobert", "en", "clinical/models")\
      .setInputCols(["sentence_embeddings"]) \
      .setOutputCol("class")

ade_clf_pipeline = Pipeline(
    stages=[documentAssembler, 
            tokenizer,
            bert_embeddings,
            ade_ner,
            ner_converter,
            assertion_ner_converter,
            biobert_assertion,
            embeddingsSentence,
            classsifierdl])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_clf_model = ade_clf_pipeline.fit(empty_data)

ade_ner_clf_pipeline = LightPipeline(ade_ner_clf_model)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[ | ][OK!]
ner_ade_biobert download started this may take some time.
[ | ][OK!]
assertion_dl_biobert download started this may take some time.
[ | ][OK!]
classifierdl_ade_conversational_biobert download started this may take some time.
Approximate size to download 21.8 MB
[ | ][OK!]

In [0]:
classsifierdl.getStorageRef()

Out[116]: 'biobert_pubmed_base_cased'

In [0]:
text =  'I have always felt tired, but no blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldnt find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead.'

light_result = ade_ner_clf_pipeline.fullAnnotate(text)

print (light_result[0]['class'][0].metadata)

chunks = []
entities = []
begin =[]
end = []
confidence = []

for n in light_result[0]['ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    confidence.append(n.metadata['confidence']) 

import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                    'begin': begin, 'end': end, "confidence":confidence})

df

{'sentence': '0', 'False': '0.028450416', 'True': '0.97154963'}
Out[117]:

,chunks,entities,begin,end,confidence
0,felt tired,ADE,14,23,0.86925
1,blood clots,ADE,33,43,0.93635
2,Voltaren,DRUG,55,62,0.9977
3,stroke,ADE,116,121,0.831
4,blood clots that traveled to my eye,ADE,131,165,0.8084857
5,voltaren,DRUG,307,314,0.7912


In [0]:
import pandas as pd

text = 'I have always felt tired, but no blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldnt find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead.'

print (text)

light_result = ade_ass_ner_model_lp_bert.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]
confidence = []

for n,m in zip(light_result['ass_ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
    confidence.append(m.metadata['confidence']) 
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, "confidence":confidence})

df

I have always felt tired, but no blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldnt find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead.
Out[118]:

,chunks,entities,assertion,confidence
0,tired,ADE,present,0.9998
1,blood clots,ADE,absent,1.0
2,stroke,ADE,present,1.0
3,blood clots that traveled to my eye,ADE,present,1.0
4,completley healthy,ADE,present,1.0


In [0]:
result = ade_ner_clf_pipeline.annotate('I just took an Advil 100 mg and it made me drowsy')

print (result['class'])
print(list(zip(result['token'],result['ner'])))

['False']
[('I', 'O'), ('just', 'O'), ('took', 'O'), ('an', 'O'), ('Advil', 'B-DRUG'), ('100', 'O'), ('mg', 'O'), ('and', 'O'), ('it', 'O'), ('made', 'O'), ('me', 'O'), ('drowsy', 'B-ADE')]

In [0]:
ade_ner_clf_model.write().overwrite().save('/databricks/driver/ade_pretrained_pipeline')

In [0]:
from sparknlp.pretrained import PretrainedPipeline

ade_pipeline = PretrainedPipeline.from_disk('/databricks/driver/ade_pretrained_pipeline')

ade_pipeline.annotate('I just took an Advil 100 mg and it made me drowsy')

Out[121]: {'ner_chunk': ['Advil', 'drowsy'],
 'assertion': ['conditional'],
 'sentence_embeddings': ['I just took an Advil 100 mg and it made me drowsy'],
 'token': ['I',
 'just',
 'took',
 'an',
 'Advil',
 '100',
 'mg',
 'and',
 'it',
 'made',
 'me',
 'drowsy'],
 'ner': ['O', 'O', 'O', 'O', 'B-DRUG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ADE'],
 'class': ['False'],
 'ass_ner_chunk': ['drowsy'],
 'embeddings': ['I',
 'just',
 'took',
 'an',
 'Advil',
 '100',
 'mg',
 'and',
 'it',
 'made',
 'me',
 'drowsy'],
 'sentence': ['I just took an Advil 100 mg and it made me drowsy']}

In [0]:
ade_pipeline.model.stages

Out[122]: [DocumentAssembler_dd3862178317,
 REGEX_TOKENIZER_3a7fa38273c4,
 BERT_EMBEDDINGS_c6741c518b81,
 MedicalNerModel_4fc5b46ae2cf,
 NerConverter_7d7f401ee30d,
 NerConverter_d5c9abbcca4a,
 ASSERTION_DL_2f4db8443148,
 SentenceEmbeddings_398a0af21f81,
 ClassifierDLModel_6edc7e323980]

## Pretrained ADE Pipeline

A pipeline for `Adverse Drug Events (ADE)` with `ner_ade_healthcare`, and `classifierdl_ade_biobert`. It will extract `ADE` and `DRUG` clinical entities, and then assign ADE status to a text(`True` means ADE, `False` means not related to ADE). Also extracts relations between `DRUG` and `ADE` entities (`1` means the adverse event and drug entities are related, `0` is not related).

In [0]:
pretrained_ade_pipeline = PretrainedPipeline('explain_clinical_doc_ade', 'en', 'clinical/models')

explain_clinical_doc_ade download started this may take some time.
Approx size to download 462.2 MB
[ | ][OK!]

In [0]:
pretrained_ade_pipeline.model.stages

Out[124]: [DocumentAssembler_c6ef83f00cc3,
 REGEX_TOKENIZER_08736789f5bd,
 BERT_EMBEDDINGS_c6741c518b81,
 SentenceEmbeddings_d06364b60c7d,
 ClassifierDLModel_6edc7e323980,
 MedicalNerModel_4fc5b46ae2cf,
 NerConverterInternal_6d6441231267,
 POS_6f55785005bf,
 dependency_d5a8da6c9093,
 RelationExtractionModel_6a65c9992836,
 NerConverterInternal_5ec5840d7eaf,
 ASSERTION_DL_2f4db8443148]

In [0]:
result = pretrained_ade_pipeline.fullAnnotate("The main adverse effects of Leflunomide consist of diarrhea, nausea, liver enzyme elevation, hypertension, alopecia, and allergic skin reactions.")

result[0].keys()

Out[125]: dict_keys(['bert_sentence_embeddings', 'bert_embeddings', 'document', 'ner_chunks_ade_assertion', 'ner_tags_ade', 'relations_ade_drug', 'ner_chunks_ade', 'assertion_ade', 'tokens', 'class', 'pos_tags', 'dependencies'])

In [0]:
result[0]['class'][0].metadata

Out[126]: {'sentence': '0', 'False': '0.0033158972', 'True': '0.99668413'}

In [0]:
text = "I experienced fatigue, muscle cramps, anxiety, agression and sadness after taking Lipitor but no more adverse after passing Zocor."

import pandas as pd

chunks = []
entities = []
begin =[]
end = []
confidence=[]

print ('sentence:', text)
print()

result = pretrained_ade_pipeline.fullAnnotate(text)

print ('ADE status:', result[0]['class'][0].result)

print ('prediction probability>> True : ', result[0]['class'][0].metadata['True'], \
        'False: ', result[0]['class'][0].metadata['False'])

for n in result[0]['ner_chunks_ade']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    confidence.append(n.metadata["confidence"])

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                'begin': begin, 'end': end,"confidence":confidence})

df


sentence: I experienced fatigue, muscle cramps, anxiety, agression and sadness after taking Lipitor but no more adverse after passing Zocor.

ADE status: True
prediction probability>> True : 0.9994361 False: 5.638876E-4
Out[127]:

,chunks,entities,begin,end,confidence
0,fatigue,ADE,14,20,0.971
1,muscle cramps,ADE,23,35,0.99435
2,anxiety,ADE,38,44,0.9997
3,agression,ADE,47,55,0.9983
4,sadness,ADE,61,67,0.5481
5,Lipitor,DRUG,82,88,0.9982
6,Zocor,DRUG,124,128,0.9951


#### with AssertionDL

In [0]:
import pandas as pd

text = """I have an allergic reaction to vancomycin. My skin has be itchy, sore throat/burning/itchy, and numbness in tongue and gums. 
I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication."""

print (text)

light_result = pretrained_ade_pipeline.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]
confidence=[]


for n,m in zip(light_result['ner_chunks_ade_assertion'],light_result['assertion_ade']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
    confidence.append(m.metadata["confidence"])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, "confidence":confidence})

df

I have an allergic reaction to vancomycin. My skin has be itchy, sore throat/burning/itchy, and numbness in tongue and gums. 
I would not recommend this drug to anyone, especially since I have never had such an adverse reaction to any other medication.
Out[128]:

,chunks,entities,assertion,confidence
0,allergic reaction,ADE,present,0.998
1,itchy,ADE,present,0.8414
2,sore throat/burning/itchy,ADE,present,0.9019
3,numbness in tongue and gums,ADE,present,0.9991


#### with Relation Extraction

In [0]:
import pandas as pd

text = """A patient had undergone a renal transplantation as a result of malignant hypertension, and immunosuppressive therapy consisting of cyclosporin and prednisone ,  developed  sweating  and  thrombosis alone 5 years following the transplantation but there were not stomach pain 
"""
 
print (text)

results = pretrained_ade_pipeline.fullAnnotate(text)

rel_pairs=[]

for rel in results[0]["relations_ade_drug"]:
    rel_pairs.append((
        rel.result, 
        rel.metadata['entity1'], 
        rel.metadata['entity1_begin'],
        rel.metadata['entity1_end'],
        rel.metadata['chunk1'], 
        rel.metadata['entity2'],
        rel.metadata['entity2_begin'],
        rel.metadata['entity2_end'],
        rel.metadata['chunk2'], 
        rel.metadata['confidence']
    ))

rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])
rel_df

A patient had undergone a renal transplantation as a result of malignant hypertension, and immunosuppressive therapy consisting of cyclosporin and prednisone , developed sweating and thrombosis alone 5 years following the transplantation but there were not stomach pain 

Out[129]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,1,DRUG,131,141,cyclosporin,ADE,172,179,sweating,0.999998
1,1,DRUG,131,141,cyclosporin,ADE,187,196,thrombosis,1.0
2,1,DRUG,147,156,prednisone,ADE,172,179,sweating,0.9243418
3,1,DRUG,147,156,prednisone,ADE,187,196,thrombosis,0.9999062


In [0]:
from sparknlp_display import RelationExtractionVisualizer

vis = RelationExtractionVisualizer()

re_vis = vis.display(results[0], 'relations_ade_drug', show_relations=True,return_html=True) # default show_relations: True
displayHTML(re_vis)



/databricks/python/lib/python3.8/site-packages/sparknlp_display/relation_extraction.py:366: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
 relation_coordinates = np.array(relation_coordinates)

A patient had undergone a renal transplantation as a result of malignant hypertension, and immunosuppressive therapy consisting of cyclosporin DRUG and prednisone DRUG , developed sweating ADE and thrombosis ADE alone 5 years following the transplantation but there were not stomach pain <polyline fill="none" points="649.0,155.0 651.4795918367346,153.7960798646822 653.9591836734694,152.59312871337622 656.4387755102041,151.39211553009378 658.9183673469388,150.19400929884657 661.3979591836735,148.99977900364644 663.8775510204081,147.81039362850512 666.357142857143,146.62682215743445 668.8367346938776,145.45003357444605 671.3163265306123,144.28099686355176 673.7959183673469,143.12068100876337 676.2755102040815,141.97005499409258 678.7551020408163,140.83008780355124 681.234693877551,139.70174842115105 683.7142857142857,138.5860058309038 686.1938775510205,137.48382901682123 688.6734693877552,136.39618696291512 691.1530612244899,135.32404865319725 693.6326530612245,134.2683830716793 696.1122448979592,133.23015920237316 698.5918367346937,132.2103460292905 701.0714285714284,131.20991253644314 703.5510204081633,130.22982770784282 706.0306122448981,129.2710605275013 708.5102040816327,128.33457997943034 710.9897959183673,127.42135504764173 713.4693877551019,126.53235471614718 715.9489795918366,125.6685479689585 718.4285714285716,124.83090379008749 720.9081632653061,124.02039116354581 723.3877551020407,123.2379790733453 725.8673469387754,122.48463650349771 728.3469387755101,121.76133243801479 730.8265306122449,121.06903586090831 733.3061224489795,120.40871575619002 735.7857142857142,119.78134110787173 738.265306122449,119.18788089996517 740.7448979591836,118.62930411648207 743.2244897959183,118.10657974143427 745.704081632653,117.62067675883348 748.1836734693877,117.17256415269148 750.6632653061224,116.76321090702004 753.1428571428571,116.39358600583091 755.6224489795918,116.06465843313586 758.1020408163265,115.77739717294665 760.5816326530612,115.53277120927505 763.0612244897959,115.33174952613282 765.5408163265306,115.17530110753174 768.0204081632653,115.06439493748353 770.5,115.0 770.5,115.0 772.9795918367346,114.9827622844223 775.4591836734692,115.01203580140927 777.9387755102041,115.08685156694916 780.4183673469387,115.20624059703015 782.8979591836735,115.36923390764053 785.3775510204081,115.57486251476851 787.857142857143,115.82215743440234 790.3367346938776,116.11014968253025 792.8163265306124,116.43787027514047 795.2959183673469,116.80435022822121 797.7755102040816,117.20862055776078 800.2551020408162,117.64971227974738 802.7346938775509,118.12665641016923 805.2142857142857,118.63848396501459 807.6938775510206,119.18422596027166 810.1734693877553,119.7629134119287 812.6530612244899,120.37357733597398 815.1326530612245,121.01524874839565 817.6122448979592,121.68695866518203 820.0918367346937,122.3877381023213 822.5714285714286,123.11661807580174 825.0510204081633,123.87262960161156 827.530612244898,124.654803695739 830.0102040816327,125.46217137417233 832.4897959183675,126.29376365289971 834.9693877551019,127.14861154790944 837.4489795918367,128.02574607518974 839.9285714285714,128.92419825072886 842.4081632653063,129.842999090515 844.8877551020408,130.7811796105364 847.3673469387757,131.73777082678137 849.8469387755101,132.71180375523804 852.3265306122449,133.7023094118947 854.8061224489795,134.7083188127396 857.2857142857144,135.72886297376095 859.765306122449,136.76297291094698 862.2448979591836,137.80967964028594 864.7244897959183,138.86801417776604 867.204081632653,139.9370075393756 869.6836734693877,141.01569074110276 872.1632653061224,142.1030947989358 874.6428571428571,143.19825072886297 877.1224489795918,144.30018954687245 879.6020408163265,145.40794226895255 882.0816326530612,146.52053991109145 884.5612244897959,147.63701348927742 887.0408163265306,148.75639401949866 889.5204081632653,149.87771251774345 892.0,151.0" stroke="#6e5772" stroke-width="1" /> 1 <polyline fill="none" points="478.0,155.0 482.2244897959183,153.79607

You can check the links below if you want to see more examples of; 

- Pretrained Clinical Pipelines in Spark NLP : [1. Detect Clinical Entities, Relations and Assertion Status with Pretrained Pipelines](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.Pretrained_Clinical_Pipelines.ipynb)
- Relation Extraction Model of DRUG and ADE entities `re_ade_biobert`: [4. Training and Reusing Clinical Relation Extraction Models](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/products/databricks/healthcare/4.%20Training%20and%20Reusing%20Clinical%20Relation%20Extraction%20Models.ipynb)

End of Notebook #